# Under the Hood of t-SNE

<p align="center">
    <img src="Assets/under_the_hood.png" alt="drawing" width="600"/>
</p>

The goal of this tutorial is to provide a general understanding of *t-Distributed Stochastic Neighbor Embedding* (t-SNE).
The notebook follows the development described in the original t-SNE paper :

> [1] L. Van der Maaten and G. Hinton, ‘Visualizing data using t-SNE.’, 
>    Journal of machine learning research, vol. 9, no. 11, 2008, 
>    Accessed: Apr. 03, 2024. [Online]. 
>    Available: https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf?fbcl

The visuals are heavily inspired by multiple sources [2,3] and the code is a simplified version of the scikit-learn implementation [4] to understand the main idea behind t-SNE.

# Table of Contents


* 1. [Problem Formulation](#problemformulation)
* 2. [First let's understand Stochastic Neighbor Embedding (SNE)](#sNE)
    * 2.1 [Visual Representation of Similarities](#visual_similarity)
    * 2.2 [Mathematical Representation of Similarities](#math_similarity)
* 3. [What are we optimizing?](#optimization)
    * 3.1 [Cost Function C](#cost_c)
    * 3.2 [Gradient Descent to Optimize $Y = \{y_1, y_2, ..., y_n\}$](#gradient_descent)
* 4. [What About $\sigma_i$?](#sigma)
    * 4.1 [How should we select this parameter?](#sigma_selection)
    * 4.2 [Perplexity](#perplexity)
    * 4.3 [Perplexity of a Probability Model](#perplexity_def)
    * 4.4 [Intuition for Perplexity](#perplexity_intuition)
    * 4.5 [Binary search for $\sigma_i$](#binary_search)
* 6. [t-Distributed Stochastic Neighbor Embedding](#tsne)
    * 6.1 [Symmetric SNE](#sym_tsne)
    * 6.2 [Pairwise Similarites](#paiwise_distance)
    * 6.3 [Crowding Problem](#crowding)
    * 6.4 [Mismatched Tails can Compensate for Mismatched Dimensionalities](#mismatched_tails)
* 7. [Final Algorithm](#final_alg)
* 8. [Now let's Implement t-SNE](#implement)
    * 8.1 [Random initialization of the low-dimensional representation](#init)
    * 8.2 [Matrix of euclidian distances](#distance)
    * 8.3 [Joint probabilities to represent Similarities (high-dimensional)](#joint_prob)
    * 8.4 [Gradient of the KL divergence](#gradient)
    * 8.5 [Gradient descent](#gradient_descent_code)
    * 8.6 [Compile everything in one class](#compile)
    * 8.7 [Complete code](#complete_code)
    * 8.8 [Let's test the described implementation](#test_implementation)
* 9. [References](#references)

## Problem Formulation <a class="anchor" id="problemformulation"></a>

t-SNE is a dimensionality reduction (DR) technique. This process attempts to reduce the number of dimensions associated to a dataset and retain a maximum amount of information. In this tutorial DR offers a way to transform a data matrix $X\in{\rm I\!R}^{p\times q}$ containing $p$ signals of size $q$ into a low-dimensionality representation $Y\in {\rm I\!R}^{p\times l}$ ($l<q$). 

## First let's understand Stochastic Neighbor Embedding (SNE) <a class="anchor" id="sNE"></a>

- 'Stochastic Neighbor Embedding (SNE) starts by converting the high-dimensional Euclidean distances between datapoints into conditional probabilities that represent similarities.' [1]

- 'Conditional probability, $p_{j|i}$, that $x_i$ would pick $x_j$ as its neighbor if neighbors were picked in proportion to their probability density under a Gaussian centered at $x_i$.' [1]

### Visual Representation of Similarities <a class="anchor" id="visual_similarity"></a>

Imagine a 2D toy dataset with 8 points. 4 points come from a Gaussian centered in (0,0) and 4 from a Gaussian centered in (0,1).
Let's evaluate the similarities for a single point $x_i$.

<p align="center">
    <img src="Assets/initial_distribution_focus.svg" alt="drawing" width="1000"/>
</p>

The distance is converted in a conditional probability. In the first dimension the similarity between points $x_i$ and $x_j$ resembles a simple 1D Gaussian.

<p align="center">
    <img src="Assets/gaussian.svg" alt="drawing" width="1000"/>
</p>

### Mathematical Representation of Similarities <a class="anchor" id="math_similarity"></a>

$$
\text{Conditional probability (high-dimensional) : }\quad\quad p_{j|i} = \frac{\exp\left( -||x_i - x_j||^2 / 2\sigma_i^2\right)}{\sum_{k\neq i}\exp\left( -||x_i - x_k||^2 / 2\sigma_i^2\right)}
$$

$$
\text{Conditional probability (low-dimensional) : }\quad\quad q_{j|i} = \frac{\exp\left( -||y_i - y_j||^2 \right)}{\sum_{k\neq i}\exp\left( -||y_i - y_k||^2 \right)}
$$


- $p_{j|i}$ : Similarity between high-dimensional points $x_i$ and $x_j$.
- $q_{j|i}$ : Similarity between low-dimensional points $y_i$ and $y_j$.
- $x_i$ : High-dimensional samples
- $y_i$ : Low-dimensional samples
- $\sigma_i$ : Standard deviation of the Gaussians in high-dimensional space (set to $1/\sqrt{2}$ for low-dimensional space)


Note : $p_{i|i} = 0$ and $q_{i|i} = 0$, since the focus is pairwise similarities.

## What are we optimizing? <a class="anchor" id="optimization"></a>

Typically dimensionality reduction techniques can be formulated in terms of an optimization problem. The question becomes: What process can use the initial data $X$ to obtain this ideal low-dimensional representation $Y$? For SNE we 'minimize the sum of the Kullback-Leibler divergences (also called relative entropy) over all data points using a gradient descent'[1]. This metric is a 'statistical distance that measure of how one probability distribution $P$ is different from a second reference probability distribution $Q$'[5].

### Cost Function C <a class="anchor" id="cost_c"></a>

Formally the cost function is defined as :

$$
C = \sum_i KL(P_i||Q_i) = \sum_i \sum_j p_{j|i}\log \frac{p_{j|i}}{q_{j|i}}
$$

### Gradient Descent to Optimize $Y = \{y_1, y_2, ..., y_n\}$ <a class="anchor" id="gradient_descent"></a>

The gradient descent optimizes the cost function using the following iterative step. $Y$ can be initialized using an arbitrary process like random points or principal component analysis (PCA).

$$
Y^{(t)} = Y^{(t-1)} + \eta \frac{\partial C}{\partial y_i} +\alpha(t) (Y^{(t-1)} - Y^{(t-2)})
$$

Where

$$
\frac{\partial C}{\partial y_i} = 2 \sum_j (p_{j|i} - q_{j|i} + p_{i|j} - q_{i|j})(y_i - y_j)
$$


- $Y^{(t)}$ : Solution at iteration $t$.
- $\eta$ : Learning rate
- $\alpha(t)$ : Momentum at iteration $t$
- $\frac{\partial C}{\partial y_i}$ : Gradient

## What About $\sigma_i$? <a class="anchor" id="sigma"></a>

One important parameter missing from the method previously described is the variance $\sigma_i^2$ used to evaluate the similarities of every high-dimensional point $x_i$. In denser regions it would be interesting to have smaller values for $\sigma_i^2$ to capture important features of the data. 

### How should we select this parameter? <a class="anchor" id="sigma_selection"></a>

An interesting way to think of the problem is by looking at the entropy associated to the similarity functions (Gaussians), the entropy increases as $\sigma_i$ increases. But is it actually easier for humans to approximate the entropy of our high-dimensional data instead of $\sigma_i$? 

Some people might think entropy is better since it is a measure of uncertainty and it is possible to think of it as the number of bits needed to encode random samples from a distribution [6]. With some effort we could estimate this entropy, although I personally find it difficult to have an intuition for this value.

<details>
  <summary><i>Proof of Entropy increasing </i></summary>

   \begin{aligned}
    H(x) & =-\int p(x) \log p(x) \mathrm{d} x \\
    & =-\mathbb{E}\left[\log \mathcal{N}\left(\mu, \sigma^2\right)\right] \\
    & =-\mathbb{E}\left[\log \left[\left(2 \pi \sigma^2\right)^{-1 / 2} \exp \left(-\frac{1}{2 \sigma^2}(x-\mu)^2\right)\right]\right] \\
    & =\frac{1}{2} \log \left(2 \pi \sigma^2\right)+\frac{1}{2 \sigma^2} \mathbb{E}\left[(x-\mu)^2\right] \\
    & = \frac{1}{2} \log \left(2 \pi \sigma^2\right)+\frac{1}{2} .
    \end{aligned}
   
   From [7]
    
</details>

This is where perplexity comes in.

### Perplexity <a class="anchor" id="perplexity"></a>

We use instead Perplexity which is defined as

$$
\text{Perplexity :}\quad\quad Perp(P_i) = 2^{H(P_i)} 
$$

$$
\text{Shannon entropy :}\quad\quad H(P_i) = -\sum_j p_{j|i} \log_2 p_{j|i} 
$$

### Perplexity of a Probability Model <a class="anchor" id="perplexity_def"></a>

If we look at the Wikipedia definition : 

'In information theory, perplexity is a measure of uncertainty in the value of a sample from a discrete probability distribution. The larger the perplexity, the less likely it is that an observer can guess the value which will be drawn from the distribution.' [8]


<details>
  <summary><i>Why does Perplexity look so familiar? </i></summary>

   Recall your Quantum information/Classical information class where we discuss Shannon's noiseless coding theorem. We typically discuss that for a string of $N$ bits we can encode $2^N$ messages (combinations). Although, given messages follow specific probability distributions, it is possible to reduce on average the number of bits needed to encode messages. It follows that the number of different typical messages that can be sent is approximately :  

   $$
    W = 2^{NH(p)}
   $$

  'Hence it is only necessary to use $NH(p)$ bits rather than $N$ bits to faithfully encode the message.' [9]


</details>

### Intuition for Perplexity <a class="anchor" id="perplexity_intuition"></a>

In the original paper for t-SNE it is explained that, 'the perplexity can be interpreted as a smooth measure of the effective number of neighbors' [1]. This is why the user defined parameter for SNE (and t-SNE) is the perplexity and not $\sigma_i$. On a graph we can see that increasing $\sigma_i$ (increasing the perplexity) increases the number of significant neighbors (effective neighbors). On the following figure, we can see how for $\sigma_i=0.1$ we have effectively zero points similar to $x_i$.

<p align="center">
    <img src="Assets/gaussian_2sigma.svg" alt="drawing" width="1000"/>
</p>

Also, an interesting fact about perplexity is that for a uniform discrete variable with $K$ outcomes, the perplexity is directly $K$.
Coming back to the selection of $\sigma_i$, it is possible to select this parameter so that the conditional probabilities given a sample $x_i$ have an uncertainty that resembles a $K$-sided die. 


<details>
  <summary><i>Proof for the perplexity of a uniform discret random variable </i></summary>

   \begin{aligned}
    \operatorname{Perplexity}(X) & :=2^{H(X)} \\
    & =2^{\frac{1}{K}-\sum_{i=1}^K \log _2 \frac{1}{K}} \\
    & =2^{-\log _2 \frac{1}{K}} \\
    & =\frac{1}{2^{\log _2 \frac{1}{K}}} \\
    & =K
    \end{aligned}

   From [8]

</details>


### Binary search for $\sigma_i$ <a class="anchor" id="binary_search"></a>

Given a user-defined perplexity, the algorithm achieves a binary search (type of search algorithm) for every sample $x_i$ in the original space and finds a value of $\sigma_i$ that matches the perplexity. To make the process faster we normally consider a user-defined number of nearest neighbor.

<details>
  <summary><i> Binary search? </i></summary>

   A binary search is a type of search algorithm that 'finds the position of a target value within a sorted array. Binary search compares the target value to the middle element of the array. If they are not equal, the half in which the target cannot lie is eliminated and the search continues on the remaining half, again taking the middle element to compare to the target value, and repeating this until the target value is found. If the search ends with the remaining half being empty, the target is not in the array.' [10]

</details>

## t-Distributed Stochastic Neighbor Embedding <a class="anchor" id="tsne"></a>

SNE gives interesting results although it has major draw backs such as :

- Difficult to optimize cost function
- 'crowding problem' (discussed later)

t-SNE attempts to minimize these challenges by introducing a symmetric conditional probability and a introduces a Student-t distribution rather than a Gaussian function to compute the low-dimensional space.


### Symmetric SNE <a class="anchor" id="sym_tsne"></a>

SNE minimizes the sum of the KL divergences between conditional probabilities $p_{i|j}$ and $q_{i|j}$

$$
C = \sum_i KL(P_i||Q_i) = \sum_i \sum_j p_{j|i}\log \frac{p_{j|i}}{q_{j|i}}
$$

Instead symmetric SNE minimizes a single KL divergence between a joint probability $P$ and $Q$.

$$
C = KL(P||Q) = \sum_i \sum_j p_{ij}\log \frac{p_{ij}}{q_{ij}}
$$

Again $p_{ii}=0$ and $q_{ii}=0$ although $p_{ij}=p_{ji}$ and $q_{ij}=q_{ji}$.

### Pairwise Similarites <a class="anchor" id="paiwise_distance"></a>

$$
\text{high-dimensional : }\quad\quad p_{ij} = \frac{p_{j|i}+p_{i|j}}{2n}
$$

$$
\text{low-dimensional : }\quad\quad q_{ij} = \frac{\exp\left( -||y_i - y_j||^2 \right)}{\sum_{k\neq i}\exp\left( -||y_i - y_k||^2 \right)}
$$

This increases the impact of outliers that normally would have little impact on the cost function by guaranteeing that $\sum_j p_{ij} > 1/2n$ for all datapoints $x_i$.

### Crowding Problem <a class="anchor" id="crowding"></a>

Sometimes it is impossible to create a faithful representation of a high-dimensional space simply by considering distance. We can imagine data embedded in a manifold that has a number of dimensional intrinsically higher than the umber of dimension used for the DR output. A simple example would be to represent the neighbors of points that form a equilateral triangle inside a one-dimensional space.


<p align="center">
    <img src="Assets/triangle.svg" alt="drawing" width="300"/>
    <img src="Assets/line.svg" alt="drawing" width="300"/>
</p>

It does not matter how we rearrange the points inside the one-dimensional space we will always find some inconsistency with the original structure in terms of neighbors. 

Also inside the high-dimensional space there are many possibilities for points to exist moderately close to each other while being separated. The DR process will tend to place points with similar distances from clusters close to each other even if there is variety in the position inside the original space. Image points that follow a Gaussian distribution in 2 dimensions and need to be placed following a Gaussian in 1 dimension, points will have a limited amount of space to represent the complexity of the original space. 


<p align="center">
    <img src="Assets/2d_gaussian_points.svg" alt="drawing" width="300"/>
    <img src="Assets/1d_gaussian_points.svg" alt="drawing" width="300"/>
</p>


### Mismatched Tails can Compensate for Mismatched Dimensionalities <a class="anchor" id="mismatched_tails"></a>

To overcome the 'crowding problem' t-SNE uses a student t-distribution to represent the low-dimensional space which offers more space to the low-dimensional representation to separate data.

<p align="center">
    <img src="Assets/t_vs_gauss.svg" alt="drawing" width="1000"/>
</p>

The final low dimensional joint probability is defined as 

$$
q_{ij} = \frac{(1+||y_i-y_j||^2)^{-1}}{\sum_{k\neq l} (1+||y_k - y_l||^2)^{-1}}
$$



## Final Algorithm <a class="anchor" id="final_alg"></a>

Exactly like SNE a gradient descent is used to optimize the cost function. The gradient becomes :

$$
\frac{\partial C}{\partial y_i} = \sum_j(p_{ij} - q_{ij})(y_i-y_j)(1+||y_i-y_j||^2)^{-1}
$$

The final algorithm becomes :

<p align="center">
    <img src="Assets/algo.png" alt="drawing" width="1000"/>
</p>

## Now let's Implement t-SNE <a class="anchor" id="implement"></a>

### Random initialization of the low-dimensional representation <a class="anchor" id="init"></a>

$$
Y = \{y_1, y_2, ..., y_n\}
$$

In [1]:
import numpy as np

def random_init(self):
    
    # Random initialization of low-dimensional representation Y
    random_state = np.random.RandomState(42)
    Y_embedded = 1e-4 * random_state.standard_normal( 
                size = (self.n_samples, self.n_components)
            ).astype(np.float32)
    Y = Y_embedded.ravel()

    return Y

### Matrix of euclidian distances <a class="anchor" id="distance"></a>

We have a data matrix $X\in{\rm I\!R}^{p\times q}$ containing $p$ samples of size $q$.

We want to build a distance matrix $D$ with elements :

$$
D_{ij} = ||x_i - x_j||^2 = \left(\sqrt{\sum_{s=1}^q (x_{is} - x_{js})^2}\right)^2 = \sum_{s=i}^q (x_{is} - x_{js})^2
$$

<details>
  <summary><i>Idea to compute D using matrices </i></summary>

   Let's consider the initial data matrix $X$ with rows $x_i$.

   $$
    D_{ij} = (x_i - x_j)(x_i - x_j)^T = x_i \cdot x_i - 2 x_i \cdot x_j + x_j \cdot x_j 
    $$
    
</details>

Instead of evaluating the distances between every point, it is much more efficient to consider the $k$ nearest neighbors of a sample $x_i$ to compute the similarities. If the number of neighbors is adequate, the result is almost identical to the exact method. The gain in computation time can change the runtime from hours to seconds. 

In [2]:
from sklearn.neighbors import NearestNeighbors

def distances_knn(self, X : np.array):

    # Find the nearest neighbors for every point
    knn = NearestNeighbors(n_neighbors = self.n_neighbors, metric = "euclidean")
    knn.fit(X)
    distances_nn = knn.kneighbors_graph(mode = "distance")
    del knn # Free memory
    distances_nn.data **= 2

    distances_nn.sort_indices()
    n_samples = distances_nn.shape[0]
    distances_data = distances_nn.data.reshape(n_samples, -1)
    distances_data = distances_data.astype(np.float32, copy=False)

    return distances_data, distances_nn.indices, distances_nn.indptr

### Joint probabilities to represent Similarities (high-dimensional) <a class="anchor" id="joint_prob"></a>


$$
\begin{aligned}
    \text{Joint probability (high-dimensional) : }&\quad\quad p_{ij} = \frac{p_{j|i}+p_{i|j}}{2n} \\
    \text{Conditional probability (high-dimensional) : }&\quad\quad  p_{j|i} = \frac{\exp\left( -||x_i - x_j||^2 / 2\sigma_i^2\right)}{\sum_{k\neq i}\exp\left( -||x_i - x_k||^2 / 2\sigma_i^2\right)}\\
\end{aligned}
$$

In [3]:
from sklearn.manifold._utils import _binary_search_perplexity
from scipy.sparse import csr_matrix

MACHINE_EPSILON = np.finfo(np.double).eps

def _joint_probabilities_nn(
            self,
            distances : np.array, 
            indices : np.array, 
            indptr : np.array
        ) -> np.array:

    # Binary search and conditional probability evaluation (in Cython)
    conditional_P = _binary_search_perplexity(distances, self.perplexity, verbose = 0)

    # Symmetrize the joint probability distribution using sparse operations
    P = csr_matrix(
        (conditional_P.ravel(), indices, indptr),
        shape = (self.n_samples, self.n_samples),
    )
    P = P + P.T

    # Normalize the joint probability distribution
    sum_P = np.maximum(P.sum(), self.MACHINE_EPSILON)
    P /= sum_P

    return P

### Gradient of the KL divergence <a class="anchor" id="gradient"></a>

Considering the joint probabilities based on the student-t distribution

$$
\text{Joint probability (low-dimensional) :}\quad\quad q_{ij} = \frac{(1+||y_i-y_j||^2)^{-1}}{\sum_{k\neq l} (1+||y_k - y_l||^2)^{-1}}
$$

The KL divergence is never explicitly evaluated since we only evaluate the gradient descent. It is the gradient of the KL divergence that matters. 

$$
\frac{\partial C}{\partial y_i} = \sum_j(p_{ij} - q_{ij})(y_i-y_j)(1+||y_i-y_j||^2)^{-1}
$$

In [4]:
from sklearn.manifold import _barnes_hut_tsne

def _kl_divergence_bh(
        self,
        Y : np.array,
        P # compressed sparse matrix
    ):

    Y = Y.astype(np.float32, copy=False)
    Y_embedded = Y.reshape(self.n_samples, self.n_components)

    val_P = P.data.astype(np.float32, copy=False)
    neighbors = P.indices.astype(np.int64, copy=False)
    indptr = P.indptr.astype(np.int64, copy=False)

    grad = np.zeros(Y_embedded.shape, dtype=np.float32)
    # Gradient written in Cython
    error = _barnes_hut_tsne.gradient(
        val_P = val_P,
        pos_output = Y_embedded,
        neighbors = neighbors,
        indptr = indptr,
        forces = grad,
        theta = 0.5,
        n_dimensions = self.n_components,
        verbose = 0,
        dof = self.degrees_of_freedom,
        num_threads = self.num_threads,
    )

    c = 2.0 * (self.degrees_of_freedom + 1.0) / self.degrees_of_freedom
    grad = grad.ravel()
    grad *= c

    return error, grad

### Gradient descent <a class="anchor" id="gradient_descent_code"></a>

The gradient descent is done through an iterative process where the following equation is evaluated :  

$$
Y^{(t)} = Y^{(t-1)} + \eta \frac{\partial C}{\partial y_i} +\alpha(t) (Y^{(t-1)} - Y^{(t-2)})
$$

- $Y^{(t)}$ : Solution at iteration $t$.
- $\eta$ : Learning rate
- $\alpha(t)$ : Momentum at iteration $t$
- $\frac{\partial C}{\partial y_i}$ : Gradient

In [5]:
def _gradient_descent(
        self,
        Y0 : np.array,
        P , # compressed sparse matrix
        momentum : float = 0.8
    ):

    Y = Y0.copy().ravel()
    update = np.zeros_like(Y)
    gains = np.ones_like(Y)
    error = np.finfo(float).max
    best_error = np.finfo(float).max
    best_iter = i = self.it

    for i in range(self.it, self.n_iter):

        error, grad = self._kl_divergence_bh(Y = Y, P = P)

        inc = update * grad < 0.0
        dec = np.invert(inc)
        gains[inc] += 0.2
        gains[dec] *= 0.8
        np.clip(gains, 0.01, np.inf, out=gains) # min_gain = 0.01
        grad *= gains
        update = momentum * update - self.learning_rate_ * grad
        Y += update

        grad_norm = np.linalg.norm(grad)

        # Check progress of gradient descent
        if error < best_error:
            best_error = error
            best_iter = i
        elif i - best_iter > 300: # n_iter_without_progress
            break
        if grad_norm <= 1e-7: # min_grad_norm
            break

    return Y, error, i

### Compile everything in one class <a class="anchor" id="compile"></a>

In [6]:
def fit_transform(self, X : np.array) -> np.array:

    # Initialize optimization parameters
    self.initialize_params(X)

    # Compute the distances
    distances_nn, indices, indptr = self.distances_knn(X = X)
    
    # compute the joint probability distribution for the input space
    P = self._joint_probabilities_nn(
                                distances = distances_nn, 
                                indices = indices,
                                indptr = indptr
                            )

    # Random initialization of low-dimensional representation Y
    Y = self.random_init()

    # Learning schedule (part 1): do 250 iteration with lower momentum but
    # higher learning rate controlled via the early exaggeration parameter
    P *= self.early_exaggeration
    Y, self.kl_divergence_, it = self._gradient_descent(
                                        Y0 = Y, 
                                        P = P,
                                        momentum = 0.5
                                )

    # Learning schedule (part 2): disable early exaggeration and finish
    # optimization with a higher momentum at 0.8
    P /= self.early_exaggeration
    remaining = self.n_iter - self._EXPLORATION_N_ITER
    if self.it < self._EXPLORATION_N_ITER or remaining > 0:
        self.it = self.it + 1
        Y, self.kl_divergence_, self.it = self._gradient_descent(
                                        Y0 = Y, 
                                        P = P, 
                                        momentum = 0.8
                                    )

    return Y.reshape(self.n_samples, self.n_components)

### Complete code <a class="anchor" id="complete_code"></a>

The previous functions were meant to be used in a single class. This is why some functions may not run on their own and why the word `self.` is in front of so many variables (`self` represents the instance of the class). For the sake of demonstration and simplicity, many variables are not discussed and it is recommended to use the original class from the Scikit-learn library when implementing t-SNE. The goal of this simplified version is to give a general understanding of how t-SNE is implemented.

In [7]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold._utils import _binary_search_perplexity
from sklearn.manifold import _barnes_hut_tsne

# Optimized processes (not discussed)
from sklearn.utils._openmp_helpers import _openmp_effective_n_threads #Number of threads

class TSNE():
    
    def __init__(
        self,
        n_components = 2,
        perplexity = 30.0
    ):
        # Control the number of exploration iterations with early_exaggeration on
        self._EXPLORATION_N_ITER = 250
        # Control the number of iterations between progress checks
        self._N_ITER_CHECK = 50

        self.n_components = n_components
        self.perplexity = perplexity
        self.early_exaggeration = 12.0
        self.n_iter = 1000 # number of iteration
        self.it = 0 # Current iteration number

        self.n_samples = None
        self.learning_rate_ = None
        self.n_neighbors = None
        self.degrees_of_freedom = None
        self.num_threads = None

    
    def initialize_params(self, X):

        # Define number of samples
        self.n_samples = X.shape[0]
        # Always learning_rate_ = 'auto'
        self.learning_rate_ = np.maximum(self.n_samples / self.early_exaggeration / 4, 50)
        # Compute the number of nearest neighbors to find.
        self.n_neighbors = min(self.n_samples - 1, int(3.0 * self.perplexity + 1))
        # Degree of freedom in gradient descent
        self.degrees_of_freedom = max(self.n_components - 1, 1)
        # Define the number of available threads
        self.num_threads = _openmp_effective_n_threads()
        # Numerical precision
        self.MACHINE_EPSILON = np.finfo(np.double).eps


    def random_init(self):
        
        # Random initialization of low-dimensional representation Y
        random_state = np.random.RandomState(42)
        X_embedded = 1e-4 * random_state.standard_normal( 
                    size = (self.n_samples, self.n_components)
                ).astype(np.float32)
        params = X_embedded.ravel()

        return params


    def distances_knn(self, X : np.array):

        # Find the nearest neighbors for every point
        knn = NearestNeighbors(n_neighbors = self.n_neighbors, metric = "euclidean")
        knn.fit(X)
        distances_nn = knn.kneighbors_graph(mode = "distance")
        del knn # Free memory
        distances_nn.data **= 2

        distances_nn.sort_indices()
        n_samples = distances_nn.shape[0]
        distances_data = distances_nn.data.reshape(n_samples, -1)
        distances_data = distances_data.astype(np.float32, copy=False)

        return distances_data, distances_nn.indices, distances_nn.indptr
    

    def _joint_probabilities_nn(
            self,
            distances : np.array, 
            indices : np.array, 
            indptr : np.array
        ) -> np.array:

        # Binary search and conditional probability evaluation (in Cython)
        conditional_P = _binary_search_perplexity(distances, self.perplexity, verbose = 0)

        # Symmetrize the joint probability distribution using sparse operations
        P = csr_matrix(
            (conditional_P.ravel(), indices, indptr),
            shape = (self.n_samples, self.n_samples),
        )
        P = P + P.T

        # Normalize the joint probability distribution
        sum_P = np.maximum(P.sum(), self.MACHINE_EPSILON)
        P /= sum_P

        return P



    def _kl_divergence_bh(
        self,
        Y : np.array,
        P # compressed sparse matrix
    ):

        Y = Y.astype(np.float32, copy=False)
        Y_embedded = Y.reshape(self.n_samples, self.n_components)

        val_P = P.data.astype(np.float32, copy=False)
        neighbors = P.indices.astype(np.int64, copy=False)
        indptr = P.indptr.astype(np.int64, copy=False)

        grad = np.zeros(Y_embedded.shape, dtype=np.float32)
        error = _barnes_hut_tsne.gradient(
            val_P = val_P,
            pos_output = Y_embedded,
            neighbors = neighbors,
            indptr = indptr,
            forces = grad,
            theta = 0.5,
            n_dimensions = self.n_components,
            verbose = 0,
            dof = self.degrees_of_freedom,
            num_threads = self.num_threads,
        )

        c = 2.0 * (self.degrees_of_freedom + 1.0) / self.degrees_of_freedom
        grad = grad.ravel()
        grad *= c

        return error, grad



    def _gradient_descent(
        self,
        Y0 : np.array,
        P , # compressed sparse matrix
        momentum : float = 0.8
    ):

        Y = Y0.copy().ravel()
        update = np.zeros_like(Y)
        gains = np.ones_like(Y)
        error = np.finfo(float).max
        best_error = np.finfo(float).max
        best_iter = i = self.it

        for i in range(self.it, self.n_iter):

            error, grad = self._kl_divergence_bh(Y = Y, P = P)

            inc = update * grad < 0.0
            dec = np.invert(inc)
            gains[inc] += 0.2
            gains[dec] *= 0.8
            np.clip(gains, 0.01, np.inf, out=gains) # min_gain = 0.01
            grad *= gains
            update = momentum * update - self.learning_rate_ * grad
            Y += update

            grad_norm = np.linalg.norm(grad)

            # Check progress of gradient descent
            if error < best_error:
                best_error = error
                best_iter = i
            elif i - best_iter > 300: # n_iter_without_progress
                break
            if grad_norm <= 1e-7: # min_grad_norm
                break

        return Y, error, i


    def fit_transform(self, X : np.array) -> np.array:

        # Initialize optimization parameters
        self.initialize_params(X)

        # Compute the distances
        distances_nn, indices, indptr = self.distances_knn(X = X)
        
        # compute the joint probability distribution for the input space
        P = self._joint_probabilities_nn(
                                    distances = distances_nn, 
                                    indices = indices,
                                    indptr = indptr
                                )

        # Random initialization of low-dimensional representation Y
        Y = self.random_init()

        # Learning schedule (part 1): do 250 iteration with lower momentum but
        # higher learning rate controlled via the early exaggeration parameter
        P *= self.early_exaggeration
        Y, self.kl_divergence_, it = self._gradient_descent(
                                            Y0 = Y, 
                                            P = P,
                                            momentum = 0.5
                                    )

        # Learning schedule (part 2): disable early exaggeration and finish
        # optimization with a higher momentum at 0.8
        P /= self.early_exaggeration
        remaining = self.n_iter - self._EXPLORATION_N_ITER
        if self.it < self._EXPLORATION_N_ITER or remaining > 0:
            self.it = self.it + 1
            Y, self.kl_divergence_, self.it = self._gradient_descent(
                                            Y0 = Y, 
                                            P = P, 
                                            momentum = 0.8
                                        )

        return Y.reshape(self.n_samples, self.n_components)

### Let's test the described implementation <a class="anchor" id="test_implementation"></a>

Let's test the implementation on a dataset that contains signals from transition edge sensor (TES) which are photon number resolving detectors. The signals in the dataset follow a Poisson distribution with an average of $\bar{n}=2.263$ photons. The signals and the expected photon number distribution is the following :

<p align="center">
    <img src="Assets/signals.png" alt="drawing" width="600"/>
</p>

<p align="center">
    <img src="Assets/expected_distribution.png" alt="drawing" width="600"/>
</p>

The dataset contains 10 240 samples of size 350 and we will reduce the dimension of the samples in a 2 dimensional space. 

To run t-SNE we compile all the code described previously in a file `reduced_sklearn_tsne.py` and execute the TSNE class using the following code. We first use a perplexity of 10 considering or dataset contains 10 classes (10 possible photon numbers). This value is not ideal since it should be associated to a dataset with uniform class probability but it gives use a place to start looking.

```python

import matplotlib.pyplot as plt
from Dataset import dataset_TES
from reduced_sklearn_tsne import TSNE

data = dataset_TES()

perplexity = 10
model = TSNE(n_components = 2, perplexity=perplexity)
reduced = model.fit_transform(data)

with plt.style.context("seaborn-v0_8"):
    plt.figure(figsize = (10,10))
    plt.scatter(reduced[:,0], reduced[:,1], s=1)
    plt.xlabel(r'$s_1$')
    plt.ylabel(r'$s_2$')
    plt.title(f'Perplexity : {perplexity}', fontsize = 20)
    plt.show()

```

The previous code runs in a few seconds on a good computer ($\sim$ 5 seconds) and gives the following output :

<p align="center">
    <img src="Assets/tsne_output_10.png" alt="drawing" width="600"/>
</p>

We can clearly identify 9 clusters that are associated to the expected photon number distribution!

### Now lets look how perplexity changes our output

If we run the same code for multiple values of perplexity we get the following :

<p align="center">
    <img src="Assets/tsne_output_2.png" alt="drawing" width="250"/>
    <img src="Assets/tsne_output_5.png" alt="drawing" width="250"/>
    <img src="Assets/tsne_output_10.png" alt="drawing" width="250"/>
</p>

<p align="center">
    <img src="Assets/tsne_output_20.png" alt="drawing" width="250"/>
    <img src="Assets/tsne_output_50.png" alt="drawing" width="250"/>
    <img src="Assets/tsne_output_100.png" alt="drawing" width="250"/>
</p>


<p align="center">
    <img src="Assets/tsne_output_1000.png" alt="drawing" width="250"/>
    <img src="Assets/tsne_output_10000.png" alt="drawing" width="250"/>
</p>

The runtime increases with perplexity, for a value of 2 it took 4 seconds and for a perplexity of 1000 it took 77 seconds.

We can see that increasing the perplexity in this case makes the clusters denser up to a certain point where the output becomes difficult to understand. Playing around with the different parameters can give information about high-dimensional data but this is the subject of another discussion. If you still want more information, the following article does a great job at discussing the impact of t-SNE parameters :

>[11] M. Wattenberg, F. Viégas, and I. Johnson, ‘How to Use t-SNE Effectively’, Distill, vol. 1, no. 10, p. e2, Oct. 2016, doi: 10.23915/distill.00002.

## References <a class="anchor" id="references"></a>


[1] L. Van der Maaten and G. Hinton, ‘Visualizing data using t-SNE.’, Journal of machine learning research, vol. 9, no. 11, 2008, Accessed: Apr. 03, 2024. [Online]. Available: https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf?fbcl

[2] L. Schoneveld, ‘In Raw Numpy: t-SNE’, nlml. Accessed: Apr. 05, 2024. [Online]. Available: https://nlml.github.io/in-raw-numpy/in-raw-numpy-t-sne/

[3] ‘t-SNE clearly explained - Blog by Kemal Erdem’, t-SNE clearly explained - Blog by Kemal Erdem. Accessed: Apr. 05, 2024. [Online]. Available: https://erdem.pl/2020/04/t-sne-clearly-explained

[4] ‘sklearn.manifold.TSNE’, scikit-learn. Accessed: Apr. 05, 2024. [Online]. Available: https://scikit-learn/stable/modules/generated/sklearn.manifold.TSNE.html

[5] ‘Kullback–Leibler divergence’, Wikipedia. Mar. 15, 2024. Accessed: Apr. 05, 2024. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Kullback%E2%80%93Leibler_divergence&oldid=1213814981

[6] ‘Perplexity: a more intuitive measure of uncertainty than entropy’, Matthew N. Bernstein. Accessed: Apr. 06, 2024. [Online]. Available: https://mbernste.github.io/posts/perplexity/

[7] ‘Entropy of the Gaussian’. Accessed: Apr. 06, 2024. [Online]. Available: https://gregorygundersen.com/blog/2020/09/01/gaussian-entropy/

[8] ‘Perplexity’, Wikipedia. Mar. 11, 2024. Accessed: Apr. 06, 2024. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Perplexity&oldid=1213187763

[9] S. M. Barnett, Quantum information. in Oxford master series in physics Atomic, optical, and laser physics, no. 16. New York, N.Y.: Oxford University Press, 2009.

[10] ‘Binary search algorithm’, Wikipedia. Mar. 15, 2024. Accessed: Apr. 07, 2024. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Binary_search_algorithm&oldid=1213923550

[11] M. Wattenberg, F. Viégas, and I. Johnson, ‘How to Use t-SNE Effectively’, Distill, vol. 1, no. 10, p. e2, Oct. 2016, doi: 10.23915/distill.00002.

